# GPX Writer - GPS Exchange Format

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys
import time

import sys
import os

import unittest

for path in ['python', '.', '..']:
    readersPath = os.path.join(path, 'core')
    if readersPath not in sys.path:
        sys.path.extend([readersPath])

from lxml import etree
from datetime import datetime

import numpy as np

import unittest

from track import Track

## Main Class

In [2]:
class GpxWriter():
    '''GPX file - GPS Exchange Format'''

    def __init__(self, filename=None):
        '''Basic init just records the filename'''

        self.filename = filename
        self.buffer = None


    def prepare(self, track):
        '''Prepare GPX prior to being saved'''

        xsi = 'http://www.w3.org/2001/XMLSchema-instance'

        gpx = etree.Element(
            'gpx', 
             {etree.QName(xsi, 'schemaLocation'): 'http://www.topografix.com/GPX/1/0 http://www.topografix.com/GPX/1/0/gpx.xsd'},
             creator='GPS Wizard', 
             version='1.0', 
             nsmap={None: 'http://www.topografix.com/GPX/1/0', 'xsi': xsi})

        trk = etree.SubElement(gpx, 'trk')
        trkseg = etree.SubElement(trk, 'trkseg')

        formats = self.getFormats(track)

        for i in range(track.numRecords):
            trkpt = etree.SubElement(trkseg, 'trkpt')

            for attrib in ['lat', 'lon']:
                trkpt.attrib[attrib] = formats[attrib].format(track.data[attrib][i])

            for element, abbr in [('ele', 'ele'),
                                  ('time', 'ts'),
                                  ('course', 'cog'),
                                  ('speed', 'sog'),
                                  ('sat', 'sat'),
                                  ('hdop', 'hdop')]:
                if abbr in track.data:
                    subElement = etree.SubElement(trkpt, element)
                    if element == 'time':
                        subElement.text = datetime.fromtimestamp(track.data[abbr][i]).isoformat() + 'Z'
                    elif abbr in formats:
                        subElement.text = formats[abbr].format(track.data[abbr][i])
                    else:
                        subElement.text = track.data[abbr][i]

        etree.indent(gpx, space=' ' * 4)
        self.buffer = etree.tostring(gpx, pretty_print=True)


    def getFormats(self, track):
        '''Determine field formats'''

        formats = {}

        for field, info in track.fields.items():
            formats[field] = '{{:.{}f}}'.format(info['decimals'])

        return formats


    def save(self, filename=None):
        '''Save GPX to disk'''
        
        if filename:
            self.filename = filename
            
        with open(self.filename, 'wb') as f:
            f.write(self.buffer)

## Unit Tests

In [3]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    sbnFilename = os.path.join(projdir, 'sessions', '20071227', 'MIKE_G_1003053_20071227_165512_DLG.SBP')
    sbnTrack = Track(sbnFilename)

    gpxFilename = os.path.join(projdir, 'sessions', 'gpx_writer.gpx')
    gpxWriter = GpxWriter(gpxFilename)

    pc1 = time.perf_counter()
    sbnTrack.load()
    gpxWriter.prepare(sbnTrack)
    pc2 = time.perf_counter()

    print("\nTest files loaded and prepared in %0.2f seconds" % (pc2 - pc1))


Test files loaded and prepared in 0.06 seconds


In [4]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK
